In [ ]:
import pandas as pd
import numpy as np
import math
import sqlite3

from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
te = TransactionEncoder()

In [3]:
conn = sqlite3.connect('../database.db')

In [4]:
# Подгружаем таблицы из sql
data_Parts = pd.read_sql("select Id, Name from Parts;", con=conn)
data_StructuresParts = pd.read_sql("select StructureId, PartId from StructuresParts;", con=conn)
data_Structures = pd.read_sql("select Id, TypeId from Structures;", con=conn)
data_Conductors = pd.read_sql("select PartId, TypeId from Conductors;", con=conn)

In [5]:
# Объединяем в датасет
df = data_Parts.merge(data_StructuresParts, left_on='Id', right_on='PartId', how='outer').drop('PartId', axis=1)
data_Structures.rename(columns={'Id': 'Id_str'}, inplace=True)
df = df.merge(data_Structures, left_on='StructureId', right_on='Id_str').drop(labels='Id_str', axis=1)
data_Conductors.rename(columns={'PartId': 'Id'}, inplace=True)
df = pd.concat([df, data_Conductors], axis=0)

In [7]:
# Отсекаем лишнее в наименованиях опор
df['StructureId'] = df['StructureId'].str.split('_').str[0]

# Заполняем пропуски в данных
df[['Name', 'StructureId']] = df[['Name', 'StructureId']].fillna('')

# Удаляем строки дубликаты
df = df.drop_duplicates().reset_index(drop=True) 

In [8]:
# Создаем список транзакций
transactions = df.groupby(by='StructureId')['Id'].unique().to_frame().reset_index(drop=False).drop(0, axis=0)

In [9]:
# Кодируем
encoded_tr = pd.DataFrame(te.fit(transactions['Id']).transform(transactions['Id']), columns=te.columns_)

In [10]:
# Найдём наиболее частые наборы
frequent_itemsets = fpgrowth(encoded_tr, min_support=0.001, use_colnames=True, max_len=2)

In [11]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0) #[['antecedents', 'consequents', 'confidence']]
rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x)[0])
rules['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])

In [12]:
request = df[df['StructureId'] == 'А11']

In [14]:
def get_results(request):
    results = pd.DataFrame()
    for _ in range(request.shape[0]):
        # Берем Id одного элемента из запроса
        element = request.iloc[_]['Id']

        # Находим все ассоциации по этому элементу
        answer = rules[rules['antecedents'] == element]

        # Проверяем, чтобы предлагаемые элементы не содержались в запросе, сортируем Id
        answer = answer[~answer['consequents'].isin(request['Id'])].sort_values(by='consequents')

        # Находим результат по найденным предложениям, проверяем совпадение по TypeId
        result = df[(df['Id'].isin(answer['consequents'])) & (df['TypeId'] == request.iloc[_]['TypeId'])]

        # Удаляем дубликаты, сортируем по Id
        result = result.drop_duplicates(subset=['Id']).sort_values(by='Id')

        # Приводим ответ в соответствие с отфильтрованным результатом
        answer = answer[answer['consequents'].isin(result['Id'])]

        # Добавляем в таблицу с результатом метрику из ответа
        result['metric'] = list((answer['confidence']*0.55 + answer['consequent support']*0.45))
        result['request'] = list(answer['antecedents'])

        # Сортируем результат по метрике
        result = result.sort_values(by='metric', ascending=False)

        # Удаляем из результата повторения по id в общих результатах
        try:
            result = result[~result['Id'].isin(results['Id'])]
        except:
            pass

        # Берем верхние строки
        result = result[:math.ceil(25 / request.shape[0])]

        # Добавляем результат по одному элементу в общие результаты
        results = pd.concat([results, result], axis=0)

    results = results.sort_values(by='metric', ascending=False).head(25).sort_values(by='Id')
    
    return results

In [15]:
get_results(request)

,Id,Name,StructureId,TypeId,metric,request
19518,COT36,Скрепа,Pole,support,0.171717,SL37.2
19519,COT37,"Лента бандажная стальная 19х0,75 мм",Pole,support,0.171717,SL37.2
2782,LUG6-50/12LVTIN,Наконечник кабельный AL/Cu с болтами со срывно...,ККМкв вар.3,support,0.258639,COT37.2
2760,P 72,Зажим для подкл. абонента к изолир. магистраль...,А23-к-каб,support,0.224078,ЗП6
2761,PA 1500,Анкерный клиновый зажим. Cечение жилы 50-70 мм2,А23-к-каб,support,0.184939,СВ95-3
19521,SL37.1,Зажим соединительный плашечный; магистраль: 6-...,Pole,support,0.177820,SH702R
19964,SM2.21,Зажим плашечный соединительный (медь-алюминий)...,К2,support,0.182818,SLIP22.1
20206,SO130.02,Зажим поддерживающий 2-4х(25-50) на угловых оп...,ОА2,support,0.164769,SLIP22.127
19965,SO234S,Зажим анкерный 4х50/4х70/4х95/4х120,К2,support,0.276869,SH702R
19966,SOT21.01R,"Крюк сквозной M20, L=240 мм",К2,support,0.156142,SLIP22.127


In [19]:
def overlap_checking(request, batch=0.33):
    overlap = []
    lenght = len(request)
    lenght_btch = math.ceil(lenght * batch)
    for i in range(0, lenght, lenght_btch):
        request_part = request[i:i+lenght_btch]
        results = get_results(request_part)['Id']
        
        # Элементы, которых нет в запросе, ожидаемый ответ
        actual = request[~request['Id'].isin(request_part['Id'])]['Id']
        
        # Находим общие элементы в ответе и в ожидаемом ответе
        common_elements = set(actual) & set(results) 
        overlap.append(len(common_elements) / len(actual))
        
    return np.mean(overlap)

In [20]:
overlap_checking(request)

0.8583333333333334

In [21]:
# Проход по всем корзинам (7 минут)
overlaps = []

for s in df['StructureId'].unique():
    
    request = df[df['StructureId'] == s]
    
    try: overlaps.append(overlap_checking(request, batch=0.3))
    except: pass
    
np.mean(overlaps)

0.7902198849088709